# compile posterior for the BGS catalogs

In [1]:
import os
import h5py
import numpy as np
import astropy.table as aTable

In [2]:
import healpy


Bad key text.latex.preview in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95 

## compile BGS Bright

In [4]:
dir_dat = '/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/'
_bgs_n = aTable.Table.read(os.path.join(dir_dat, 'BGS_BRIGHT_N_clustering.dat.fits'))
_bgs_s = aTable.Table.read(os.path.join(dir_dat, 'BGS_BRIGHT_S_clustering.dat.fits'))

bgs_bright = aTable.vstack([_bgs_n, _bgs_s])

In [5]:
len(bgs_bright)

143853

In [6]:
bgs_bright[:5]

RA,DEC,TARGETID,NTILE,TILES,Z,COMP_TILE,rosette_number,rosette_r,FRACZ_TILELOCID,BITWEIGHTS [2],PROB_OBS,WEIGHT_ZFAIL,WEIGHT,flux_g_dered,flux_r_dered,flux_z_dered,flux_w1_dered,flux_w2_dered,NZ,WEIGHT_FKP
float64,float64,int64,int64,bytes43,float64,float64,float64,float64,float64,int64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float64
191.20458196461243,60.31535787110577,39633390185481345,1,100,0.08969256109731599,0.64,3.0,1.613220114177946,1.0,-1 .. -1,1.0,1.0,1.0,11.618794,18.39226,20.892622,13.421043,9.27371,0.04778151135868713,0.0030821516772042766
187.04622815471808,60.82670180284912,39633396124617897,1,100,0.37400455377945924,0.64,3.0,1.6801915534232357,1.0,-1 .. -1,1.0,1.0,1.0,4.046807,16.82531,38.79777,85.02852,54.11833,0.0009577827729000268,0.133626516949366
187.83760708675698,60.38987543677365,39633393155049303,1,100,0.1731108191825362,0.64,3.0,1.7267282368438097,0.5,6932875859169517021 .. -6858338461939850770,0.4609375,1.0,2.15,13.26433,27.728409,46.020153,37.03337,25.867756,0.01270745015908088,0.01149149450827468
187.05117249536465,62.614542270156655,39633416324383107,1,100,0.2703579310142208,0.64,3.0,1.5579854421783794,1.0,-1 .. -1,1.0,1.0,1.0,4.4575434,17.798513,37.420948,46.075268,30.963612,0.004051015958871262,0.035183206585599834
191.14757099005186,60.345815959235,39633390185480938,1,100,0.2819459922463374,0.64,3.0,1.5743490921284766,0.5,4864741233190265570 .. 434972757299228716,0.359375,1.0,2.74468085106383,6.782468,16.732075,29.996437,31.939342,25.343126,0.004051015958871262,0.035183206585599834


In [7]:
bgs_bright['phi'] = np.radians(bgs_bright['RA'])
bgs_bright['theta'] = np.radians(90. - bgs_bright['DEC'])
bgs_bright['healpix'] = healpy.ang2pix(64, bgs_bright['theta'], bgs_bright['phi'], nest=True)

get all healpixs

In [8]:
tiles_fuji = aTable.Table.read('/global/cfs/cdirs/desi/spectro/redux/fuji/healpix/tilepix.fits')
is_bright = (tiles_fuji['PROGRAM'] == 'bright')
is_sv3 = (tiles_fuji['SURVEY'] == 'sv3')

hpixs = np.unique(np.sort(np.array(tiles_fuji['HEALPIX'][is_bright & is_sv3])))

In [15]:
mstars = np.tile(-999., (len(bgs_bright), 30))
zmaxes = np.repeat(-999., len(bgs_bright))
theta_bfs = np.tile(-999., (len(bgs_bright), 13))

for hpix in hpixs: 
    try: 
        f = h5py.File('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/provabgs-sv3-bright-%s.BGS_BRIGHT.hdf5' % hpix, 'r')
        tid = f['targetid'][...]
        mcmc = f['samples'][...][:,-1,:,0]
        zmax = f['zmax'][...]
        ttbf = f['theta_bf'][...]
    except IndexError: 
        print('no BGS BRIGHT galaxies in %i' % hpix)
        continue
    except KeyError: 
        print('no BGS BRIGHT galaxies in %i' % hpix)
        continue
    except OSError: 
        print('no posteriors for %i' % hpix)
        continue
    except: 
        print('problem reading %i' % hpix)
        continue 
    
    is_hpix = (bgs_bright['healpix'] == hpix)

    for i in np.arange(len(bgs_bright))[is_hpix]: 
        is_target = (tid == bgs_bright['TARGETID'][i])

        if np.sum(is_target) == 1:
            mstars[i,:] = mcmc[is_target]
            zmaxes[i] = zmax[is_target]
            theta_bfs[i,:] = ttbf[is_target] 

no BGS BRIGHT galaxies in 9932
no posteriors for 9985
no posteriors for 10520
no posteriors for 25939
no posteriors for 26087


In [16]:
bgs_bright['logM_mcmc'] = mstars
bgs_bright['zmax'] = zmaxes
bgs_bright['theta_bf'] = theta_bfs

In [17]:
print("%i galaxies don't have posteriors" % np.sum(mstars[:,0] == -999.))

1341 galaxies don't have posteriors


In [18]:
bgs_bright[:5]

RA,DEC,TARGETID,NTILE,TILES,Z,COMP_TILE,rosette_number,rosette_r,FRACZ_TILELOCID,BITWEIGHTS [2],PROB_OBS,WEIGHT_ZFAIL,WEIGHT,flux_g_dered,flux_r_dered,flux_z_dered,flux_w1_dered,flux_w2_dered,NZ,WEIGHT_FKP,phi,theta,healpix,logM_mcmc [30],zmax,theta_bf [13]
float64,float64,int64,int64,bytes43,float64,float64,float64,float64,float64,int64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float64,float64,float64,int64,float64,float64,float64
191.20458196461243,60.31535787110577,39633390185481345,1,100,0.08969256109731599,0.64,3.0,1.613220114177946,1.0,-1 .. -1,1.0,1.0,1.0,11.618794,18.39226,20.892622,13.421043,9.27371,0.04778151135868713,0.0030821516772042766,3.3371495001818547,0.5180947424254233,11226,9.311141344469839 .. 9.30371384223851,0.244537973921384,9.299830413616577 .. 0.12427289814965596
187.04622815471808,60.82670180284912,39633396124617897,1,100,0.37400455377945924,0.64,3.0,1.6801915534232357,1.0,-1 .. -1,1.0,1.0,1.0,4.046807,16.82531,38.79777,85.02852,54.11833,0.0009577827729000268,0.133626516949366,3.264572534736348,0.5091701072064087,11237,11.532893592539585 .. 11.652536919740712,0.5180062804791324,11.529095176742617 .. 0.3710937098062413
187.83760708675698,60.38987543677365,39633393155049303,1,100,0.1731108191825362,0.64,3.0,1.7267282368438097,0.5,6932875859169517021 .. -6858338461939850770,0.4609375,1.0,2.15,13.26433,27.728409,46.020153,37.03337,25.867756,0.01270745015908088,0.01149149450827468,3.2783847027313433,0.5167941655539476,11237,10.562881907340254 .. 10.51416114213185,0.3653015609857238,10.502975152735333 .. 0.2744993206027024
187.05117249536465,62.614542270156655,39633416324383107,1,100,0.2703579310142208,0.64,3.0,1.5579854421783794,1.0,-1 .. -1,1.0,1.0,1.0,4.4575434,17.798513,37.420948,46.075268,30.963612,0.004051015958871262,0.035183206585599834,3.264658829759971,0.47796640455149814,11251,11.164636307948935 .. 11.122322575621661,0.3243951220160425,11.16028636319183 .. 0.6710591095679554
191.14757099005186,60.345815959235,39633390185480938,1,100,0.2819459922463374,0.64,3.0,1.5743490921284766,0.5,4864741233190265570 .. 434972757299228716,0.359375,1.0,2.74468085106383,6.782468,16.732075,29.996437,31.939342,25.343126,0.004051015958871262,0.035183206585599834,3.3361544709660023,0.5175631485037057,11226,11.088944348844176 .. 11.20103049582311,0.42353546824905075,11.14249424405227 .. 0.3651476796540595


In [19]:
bgs_bright.write('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/BGS_BRIGHT_clustering.sv3.logM.hdf5', overwrite=True)

/global/homes/c/chahah/.conda/envs/gqp/lib/python3.7/site-packages/astropy/io/misc/hdf5.py:283: UserWarning: table path was not set via the path= argument; using default path __astropy_table__
  "using default path {}".format(path))


# compile BGS Faint

In [20]:
dir_dat = '/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/'
_bgs_n = aTable.Table.read(os.path.join(dir_dat, 'BGS_FAINT_N_clustering.dat.fits'))
_bgs_s = aTable.Table.read(os.path.join(dir_dat, 'BGS_FAINT_S_clustering.dat.fits'))

bgs_faint = aTable.vstack([_bgs_n, _bgs_s])
print('%i BGS Faint galaxies' % len(bgs_faint))

FileNotFoundError: [Errno 2] No such file or directory: '/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_FAINT_N_clustering.dat.fits'

In [ ]:
bgs_faint['phi'] = np.radians(bgs_faint['RA'])
bgs_faint['theta'] = np.radians(90. - bgs_faint['DEC'])
bgs_faint['healpix'] = healpy.ang2pix(64, bgs_faint['theta'], bgs_faint['phi'], nest=True)

In [ ]:
mstars = np.tile(-999., (len(bgs_faint), 30))
zmaxes = np.repeat(-999., len(bgs_faint))
theta_bfs = np.tile(-999., (len(bgs_faint), 13))

for hpix in hpixs: 
    try: 
        f = h5py.File('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/provabgs-sv3-bright-%s.BGS_FAINT.hdf5' % hpix, 'r')
        tid = f['targetid'][...]
        mcmc = f['samples'][...][:,-1,:,0]
        zmax = f['zmax'][...]
        ttbf = f['theta_bf'][...]
    except IndexError: 
        print('no BGS FAINT galaxies in %i' % hpix)
        continue
    except KeyError: 
        print('no BGS FAINT galaxies in %i' % hpix)
        continue
    except OSError: 
        print('no posteriors for %i' % hpix)
        continue
    except: 
        print('problem reading %i' % hpix)
        continue 
    
    is_hpix = (bgs_faint['healpix'] == hpix)

    for i in np.arange(len(bgs_faint))[is_hpix]: 
        is_target = (tid == bgs_faint['TARGETID'][i])

        if np.sum(is_target) == 1:
            mstars[i,:] = mcmc[is_target]
            zmaxes[i] = zmax[is_target]
            theta_bfs[i,:] = ttbf[is_target] 
            
bgs_faint['logM_mcmc'] = mstars
bgs_faint['zmax'] = zmaxes
bgs_faint['theta_bf'] = theta_bfs

In [ ]:
print("%i BGS FAINT galaxies don't have posteriors" % np.sum(mstars[:,0] == -999.))

In [ ]:
bgs_faint[:5]

In [ ]:
bgs_faint.write('/global/cfs/cdirs/desi/users/chahah/provabgs/svda/BGS_FAINT_clustering.sv3.logM.hdf5', overwrite=True)